# Controllable generation via RL about text-guided voice conversion


In [10]:
from importlib import reload
import textrl
reload(textrl)

from NISQA.nisqa.NISQA_model import nisqaModel

import sys
sys.path.append("/work/b0990106x/TextRL/vc") 
base_path = "/work/b0990106x/TextRL"

# args_nisqa = {
#                     "mode": "predict_file",
#                     "pretrained_model": f"{base_path}/NISQA/weights/nisqa.tar",
#                     "deg": f"{base_path}/test_nisqa_wav/example2.wav",
#                     "data_dir": None,
#                     "output_dir": f"{base_path}/NISQA/result/",
#                     "csv_file": None,
#                     "csv_deg": None,
#                     "num_workers": 0,
#                     "bs": 1,
#                     "ms_channel": None,
#                 }
# args_nisqa["tr_bs_val"] = args_nisqa["bs"]
# args_nisqa["tr_num_workers"] = args_nisqa["num_workers"]

# nisqa = nisqaModel(args_nisqa)
# prediction = nisqa.predict()
# reward = float(prediction["mos_pred"].iloc[0])
# print("Reward:",reward)

for i in range(1,10):
    try:
        args_nisqa = {
                        "mode": "predict_file",
                        "pretrained_model": f"{base_path}/NISQA/weights/nisqa.tar",
                        "deg": f"{base_path}/test_nisqa_wav/example{i}.wav",
                        "data_dir": None,
                        "output_dir": f"{base_path}/NISQA/result/",
                        "csv_file": None,
                        "csv_deg": None,
                        "num_workers": 0,
                        "bs": 1,
                        "ms_channel": None,
                    }
        args_nisqa["tr_bs_val"] = args_nisqa["bs"]
        args_nisqa["tr_num_workers"] = args_nisqa["num_workers"]

        nisqa = nisqaModel(args_nisqa)
        prediction = nisqa.predict()
        reward = float(prediction["mos_pred"].iloc[0])
        print(f"Reward of example{i}: {reward}")
    except:
        print(f"Error in example{i}")
        continue
    

Reward of example1: 4.004299640655518
Reward of example2: 4.33428955078125
Reward of example3: 4.13248348236084
Reward of example4: 4.226344585418701
Reward of example5: 3.9320855140686035
Reward of example6: 4.017142295837402
Error in example7
Reward of example8: 4.13602352142334
Reward of example9: 4.193085670471191


In [ ]:
from importlib import reload
import textrl
reload(textrl)

from textrl import TextRLEnv, TextRLActor
from NISQA.nisqa.NISQA_model import nisqaModel

import sys
sys.path.append("/work/b0990106x/TextRL/vc") 
from vc.trainer_encodec_vc_inference import get_ar_prediction_v2


class MyRLEnv(TextRLEnv):
    def get_reward(self, _, predicted_list, finish):
        reward = 0
        if finish or len(predicted_list) >= self.env_max_length:
            try:
                predicted_tokens = predicted_list[0][1:-1]
                predicted_ids = self.tokenizer.convert_tokens_to_ids([f"{u}" for u in predicted_tokens])
                # print("predicted_ids:", predicted_ids)

                decode_ar = get_ar_prediction_v2(
                    self.args_predict,
                    predicted_ids,
                    self.nar_model,
                    self.tokenizer,
                    self.nar_tokenizer,
                    self.single_src_encodec,
                    self.single_instruction,
                    self.episode_counter,
                )
                # print("decode_ar:", decode_ar)
                
                # use nisqa to get the reward
                args_nisqa = {
                    "mode": "predict_file",
                    "pretrained_model": f"{base_path}/NISQA/weights/nisqa.tar",
                    "deg": f"{base_path}/output/{ts}/example.wav",
                    "data_dir": None,
                    "output_dir": f"{base_path}/NISQA/result/",
                    "csv_file": None,
                    "csv_deg": None,
                    "num_workers": 0,
                    "bs": 1,
                    "ms_channel": None,
                }
                args_nisqa["tr_bs_val"] = args_nisqa["bs"]
                args_nisqa["tr_num_workers"] = args_nisqa["num_workers"]

                nisqa = nisqaModel(args_nisqa)
                prediction = nisqa.predict()
                reward = float(prediction["mos_pred"].iloc[0])-3.0
                print(
                    "Length of predicted_list:",
                    len(predicted_list[0]),
                    ", Reward:",
                    reward,
                )

            except Exception as e:
                print("Error:", e)
                reward = 0

        return reward

In [ ]:
observation_list = []
for i in range(data_len):
    observation_list.append(
        {
            "input": "",
            "src_encodec": all_src_encodec[i],
            "instruction": all_instruction[i],
        }
    )
    print("src_encodec:", observation_list[i]["src_encodec"][0])
    print("instruction:", all_instruction[i])

# for i in range(data_len):
#     observation_list.append({'input': "", 'src_encodec': all_src_encodec[i], 'instruction': all_instruction[i]})

src_encodec: [408, 835, 835, 798, 585, 550, 535, 535, 737, 737, 377, 556, 601, 787, 8, 99, 411, 411, 378, 937, 378, 937, 804, 838, 890, 934, 47, 438, 438, 731, 738, 133, 709, 479, 479, 479, 151, 940, 502, 906, 407, 645, 70, 208, 537, 537, 1022, 681, 723, 747, 593, 804, 681, 879, 136, 967, 233, 431, 754, 421, 182, 182, 651, 879, 887, 819, 904, 904, 887, 309, 880, 396, 754, 775, 997, 222, 336, 548, 841, 269, 479, 479, 940, 23, 56, 738, 835, 395, 206, 779, 531, 862, 931, 306, 203, 755, 369, 6, 466, 716, 948, 82, 575, 288, 556, 903, 556, 392, 796, 751, 835, 103, 25, 408, 835, 835, 339, 339, 395, 250, 706, 317, 479, 800, 960, 141, 479, 908, 801, 327, 937, 559, 708, 372, 372, 573, 437, 437, 421, 203, 739, 830, 739, 358, 830, 248, 411, 411, 112, 321, 23, 23, 185, 971, 62, 339, 461, 488, 934, 148, 373, 561, 681, 760, 531, 612, 699, 23, 967, 457, 790, 154, 906, 465, 502, 884, 479, 246, 820, 601, 309, 716, 314, 377, 309, 309, 556, 118, 99, 358, 1018, 862, 779, 62, 835, 25, 254, 254, 677, 73, 143

In [ ]:
print("observation_list:", observation_list)

observation_list: [{'input': '', 'src_encodec': [[408, 835, 835, 798, 585, 550, 535, 535, 737, 737, 377, 556, 601, 787, 8, 99, 411, 411, 378, 937, 378, 937, 804, 838, 890, 934, 47, 438, 438, 731, 738, 133, 709, 479, 479, 479, 151, 940, 502, 906, 407, 645, 70, 208, 537, 537, 1022, 681, 723, 747, 593, 804, 681, 879, 136, 967, 233, 431, 754, 421, 182, 182, 651, 879, 887, 819, 904, 904, 887, 309, 880, 396, 754, 775, 997, 222, 336, 548, 841, 269, 479, 479, 940, 23, 56, 738, 835, 395, 206, 779, 531, 862, 931, 306, 203, 755, 369, 6, 466, 716, 948, 82, 575, 288, 556, 903, 556, 392, 796, 751, 835, 103, 25, 408, 835, 835, 339, 339, 395, 250, 706, 317, 479, 800, 960, 141, 479, 908, 801, 327, 937, 559, 708, 372, 372, 573, 437, 437, 421, 203, 739, 830, 739, 358, 830, 248, 411, 411, 112, 321, 23, 23, 185, 971, 62, 339, 461, 488, 934, 148, 373, 561, 681, 760, 531, 612, 699, 23, 967, 457, 790, 154, 906, 465, 502, 884, 479, 246, 820, 601, 309, 716, 314, 377, 309, 309, 556, 118, 99, 358, 1018, 862, 779,

In [ ]:
from types import SimpleNamespace

args_predict = SimpleNamespace(
    output_path=f"{base_path}/output/{ts}/example.wav", seed=0, device="cuda"
)

env = MyRLEnv(
    ar_model,
    ar_tokenizer,
    nar_model,
    nar_tokenizer,
    args_predict,
    observation_input=observation_list,
    compare_sample=1,
)
actor = TextRLActor(env, ar_model, ar_tokenizer, ts)
agent = actor.agent_ppo(update_interval=2400, minibatch_size=256, epochs=10, lr=3e-8)

model name:  BartForConditionalGeneration
----------------------------- reset -----------------------------


In [ ]:
import logging
import os
import sys

output_log_path = f"log/log_{ts}.log"
output_file_path = f"log/output_{ts}.txt"

if not os.path.exists("log"):
    os.makedirs("log")

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

handlers = logger.handlers[:]
for handler in handlers:
    logger.removeHandler(handler)

file_handler = logging.FileHandler(output_log_path)
logger.addHandler(file_handler)

In [ ]:
import sys
import time
import pfrl

start_time = time.time()
pfrl_outdir = f"ckpt/train_{ts}"

with open(output_file_path, "w") as f:
    original_stdout = sys.stdout
    sys.stdout = f
    pfrl.experiments.train_agent_with_evaluation(
        agent,
        env,
        steps=50000,
        eval_n_steps=None,
        eval_n_episodes=2,
        train_max_episode_len=1000,
        eval_interval=5000,
        outdir=pfrl_outdir,
        logger=logger,
        use_tensorboard=True,
        checkpoint_freq=10000,
    )
    sys.stdout = original_stdout
    
print("Output has been written to", output_file_path)
print("used time: ", time.time() - start_time)


/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/pfrl/agents/ppo.py:132: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  actions = torch.tensor([b["action"] for b i

Output has been written to log/output_0506-0216.txt
used time:  1623.1468076705933


In [ ]:
agent.load(pfrl_outdir + "/best")
actor.predict(observation_list[0])

----------------------------- reset -----------------------------
Episode 159 : audio saved to /work/b0990106x/TextRL/output/0506-0216/example_save_159.wav
Length of predicted_list: 652 , Reward: -0.806640625


['v_tok_835v_tok_835v_tok_835v_tok_798v_tok_585v_tok_550v_tok_535v_tok_535v_tok_737v_tok_737v_tok_377v_tok_556v_tok_601v_tok_787v_tok_8v_tok_99v_tok_411v_tok_411v_tok_378v_tok_937v_tok_378v_tok_937v_tok_804v_tok_838v_tok_890v_tok_934v_tok_47v_tok_438v_tok_438v_tok_731v_tok_738v_tok_133v_tok_709v_tok_479v_tok_479v_tok_479v_tok_151v_tok_940v_tok_502v_tok_906v_tok_407v_tok_645v_tok_70v_tok_208v_tok_537v_tok_537v_tok_1022v_tok_681v_tok_723v_tok_747v_tok_593v_tok_804v_tok_681v_tok_879v_tok_136v_tok_967v_tok_233v_tok_431v_tok_754v_tok_421v_tok_182v_tok_182v_tok_651v_tok_879v_tok_887v_tok_819v_tok_904v_tok_904v_tok_887v_tok_309v_tok_880v_tok_396v_tok_754v_tok_775v_tok_997v_tok_222v_tok_336v_tok_548v_tok_841v_tok_269v_tok_479v_tok_479v_tok_940v_tok_23v_tok_56v_tok_738v_tok_835v_tok_395v_tok_206v_tok_779v_tok_531v_tok_862v_tok_931v_tok_306v_tok_203v_tok_755v_tok_369v_tok_6v_tok_466v_tok_716v_tok_948v_tok_82v_tok_575v_tok_288v_tok_556v_tok_903v_tok_556v_tok_392v_tok_796v_tok_751v_tok_835v_tok_10

In [ ]:
print(ts)

0506-0216


In [ ]:
# import read_pickle
# # ts = "0430-1250"
# # len of the pickle file
# dir_path = f"{base_path}/replay_buffer/{ts}"
# length = len(os.listdir(dir_path))
# for i in range(length):
#     pickle_file = f"replay_buffer_update_{i}.pkl"
#     pickle_data = read_pickle.load_pickle(file_path = f'{base_path}/replay_buffer/{ts}/{pickle_file}')
#     length_of_pickle_data = len(pickle_data[0])
#     print(f"length of {pickle_file}: {length_of_pickle_data}") 
 